# Importing Northwind Database Directly From MySQL

### Preparing working environment and importing libraries:

In [10]:
% cd /mnt/Data/Jupyter/Neo4j/

/mnt/Data/Jupyter/Neo4j


In [11]:
from py2neo import Graph
import pymysql.cursors
import pandas as pd
import auth

### Initiating connection with databases:

In [12]:
neodb = Graph(bolt=True, password=auth.password)

mysqldb = pymysql.connect(host="localhost",
                                user=auth.user,
                                password=auth.password,
                                db='Northwind')

### Defining function to issue SQL query to MySQL & return data:

In [16]:
def mysql_query(sql):
    try:
        mysqldb.connect()
        df = pd.read_sql(sql, mysqldb)
    except:
        mysqldb.close()
    return(df)

### Defining function to initialise Neo4j and executing:

In [21]:
def neo_init():
    neodb.run("MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r")

neo_init()

### Querying MySQL for Products table & issuing Cyper queries to Neo4j which iterate through the query result and insert each into Neo4j as Nodes.

In [12]:
for index, row in mysql_query("""
    SELECT *
    FROM Products
    """).iterrows():
    neodb.run("""
    CREATE (n:Product  {ProductID : $ProductID,
                        ProductName : $ProductName,
                        SupplierID : $SupplierID,
                        CategoryID : $CategoryID,
                        QuantityPerUnit : $QuantityPerUnit,
                        UnitPrice : $UnitPrice,
                        UnitsInStock : $UnitsInStock,
                        UnitsOnOrder : $UnitsOnOrder,
                        ReorderLevel : $ReorderLevel,
                        Discontinued : ($Discontinued <> 0)})
    """, parameters = {"ProductID" : row["ProductID"],
                       "ProductName" : row["ProductName"],
                       "SupplierID" : row["SupplierID"],
                       "CategoryID" : row["CategoryID"],
                       "QuantityPerUnit" : row["QuantityPerUnit"],
                       "UnitPrice" : row["UnitPrice"],
                       "UnitsInStock" : row["UnitsInStock"],
                       "UnitsOnOrder" : row["UnitsOnOrder"],
                       "ReorderLevel" : row["ReorderLevel"],
                       "Discontinued" : row["Discontinued"]})

In [13]:
for index, row in mysql_query("""
    SELECT CategoryID, CategoryName, Description
    FROM Categories
    """).iterrows():
    neodb.run("""
    CREATE (n:Category  {CategoryID : $CategoryID,
                        CategoryName : $CategoryName,
                        Description : $Description})
    """, parameters = {"CategoryID" : row["CategoryID"],
                       "CategoryName" : row["CategoryName"],
                       "Description" : row["Description"]})

In [14]:
for index, row in mysql_query("""
    SELECT SupplierID, CompanyName, ContactName, ContactTitle,
    Address, City, Region, PostalCode, Country, Phone
    FROM Suppliers
    """).iterrows():
    neodb.run("""
    CREATE (n:Supplier {SupplierID : $SupplierID,
                        CompanyName : $CompanyName,
                        ContactName : $ContactName,
                        ContactTitle : $ContactTitle,
                        Address : $Address,
                        City : $City,
                        Region : $Region,
                        PostalCode : $PostalCode,
                        Country : $Country,
                        Phone : $Phone})
    """, parameters = {"SupplierID" : row["SupplierID"],
                       "CompanyName" : row["CompanyName"],
                       "ContactName" : row["ContactName"],
                       "ContactTitle" : row["ContactTitle"],
                       "Address" : row["Address"],
                       "City" : row["City"],
                       "Region" : row["Region"],
                       "PostalCode" : row["PostalCode"],
                       "Country" : row["Country"],
                       "Phone" : row["Phone"]})

In [15]:
for index, row in mysql_query("""
    SELECT CustomerID, CompanyName, ContactName, ContactTitle,
    Address, City, PostalCode, Country, Phone
    FROM Customers
    """).iterrows():
    neodb.run("""
    CREATE (n:Customer {CustomerID : $CustomerID,
                        CompanyName : $CompanyName,
                        ContactName : $ContactName,
                        ContactTitle : $ContactTitle,
                        Address : $Address,
                        City : $City,
                        PostalCode : $PostalCode,
                        Country : $Country,
                        Phone : $Phone})
    """, parameters = {"CustomerID" : row["CustomerID"],
                       "CompanyName" : row["CompanyName"],
                       "ContactName" : row["ContactName"],
                       "ContactTitle" : row["ContactTitle"],
                       "Address" : row["Address"],
                       "City" : row["City"],
                       "PostalCode" : row["PostalCode"],
                       "Country" : row["Country"],
                       "Phone" : row["Phone"]})

In [16]:
for index, row in mysql_query("""
    SELECT OrderID, CustomerID, EmployeeID, OrderDate,
    RequiredDate, ShippedDate, ShipVia, Freight, ShipName,
    ShipAddress, ShipCity, ShipPostalCode, ShipCountry
    FROM Orders
    """).iterrows():
    neodb.run("""
    CREATE (n:Order    {OrderID : $OrderID,
                        CustomerID : $CustomerID,
                        EmployeeID : $EmployeeID,
                        OrderDate : $OrderDate,
                        RequiredDate : $RequiredDate,
                        ShippedDate : $ShippedDate,
                        ShipVia : $ShipVia,
                        Freight : $Freight,
                        ShipName : $ShipName,
                        ShipAddress : $ShipAddress,
                        ShipCity : $ShipCity,
                        ShipPostalCode : $ShipPostalCode,
                        ShipCountry : $ShipCountry})
    """, parameters = {"OrderID" : row["OrderID"],
                       "CustomerID" : row["CustomerID"],
                       "EmployeeID" : row["EmployeeID"],
                       "OrderDate" : str(row["OrderDate"]),
                       "RequiredDate" : str(row["RequiredDate"]),
                       "ShippedDate" : str(row["ShippedDate"]),
                       "ShipVia" : row["ShipVia"],
                       "Freight" : row["Freight"],
                       "ShipName" : row["ShipName"],
                       "ShipAddress" : row["ShipAddress"],
                       "ShipCity" : row["ShipCity"],
                       "ShipPostalCode" : row["ShipPostalCode"],
                       "ShipCountry" : row["ShipCountry"]})

In [17]:
for index, row in mysql_query("""
    SELECT EmployeeID, concat(TitleOfCourtesy, " ", FirstName, " ", LastName)
    AS EmployeeName, BirthDate, HireDate, Address, City, Country, Extension,
    HomePhone, IFNULL(ReportsTo, 0) AS ReportsTo
    FROM Employees
    """).iterrows():
    neodb.run("""
    CREATE (n:Employee {EmployeeID : $EmployeeID,
                        EmployeeName : $EmployeeName,
                        BirthDate : $BirthDate,
                        HireDate : $HireDate,
                        Address : $Address,
                        City : $City,
                        Country : $Country,
                        Extension : $Extension,
                        HomePhone : $HomePhone,
                        ReportsTo : $ReportsTo})
    """, parameters = {"EmployeeID" : row["EmployeeID"],
                       "EmployeeName" : row["EmployeeName"],
                       "BirthDate" : str(row["BirthDate"]),
                       "HireDate" : str(row["HireDate"]),
                       "Address" : row["Address"],
                       "City" : row["City"],
                       "Country" : row["Country"],
                       "Extension" : row["Extension"],
                       "HomePhone" : row["HomePhone"],
                       "ReportsTo" : row["ReportsTo"]})

In [18]:
for index, row in mysql_query("""
    SELECT * FROM Territories
    """).iterrows():
    neodb.run("""
    CREATE (n:Territory{TerritoryID : $TerritoryID,
                        TerritoryDescription : $TerritoryDescription,
                        RegionID : $RegionID})
    """, parameters = {"TerritoryID" : row["TerritoryID"],
                       "TerritoryDescription" : row["TerritoryDescription"],
                       "RegionID" : row["RegionID"]})

In [19]:
for index, row in mysql_query("""
    SELECT * FROM Region
    """).iterrows():
    neodb.run("""
    CREATE (n:Region   {RegionID : $RegionID,
                        RegionDescription : $RegionDescription})
    """, parameters = {"RegionID" : row["RegionID"],
                       "RegionDescription" : row["RegionDescription"]})

In [20]:
for index, row in mysql_query("""
    SELECT * FROM Shippers
    """).iterrows():
    neodb.run("""
    CREATE (n:Shipper  {ShipperID : $ShipperID,
                        CompanyName : $CompanyName,
                        Phone : $Phone})
    """, parameters = {"ShipperID" : row["ShipperID"],
                       "CompanyName" : row["CompanyName"],
                       "Phone" : row["Phone"]})

In [21]:
for index, row in mysql_query("""
    SELECT *
    FROM Order_Details
    """).iterrows():
    neodb.run("""
    MATCH (p:Product), (o:Order)
    WHERE p.ProductID = $ProductID AND o.OrderID = $OrderID
    CREATE (o)-[details:INCLUDES{
                        OrderID : $OrderID,
                        ProductID : $ProductID,
                        UnitPrice : $UnitPrice,
                        Quantity : $Quantity,
                        Discount : $Discount}]->(p)
    """, parameters = {"OrderID" : row["OrderID"],
                       "ProductID" : row["ProductID"],
                       "UnitPrice" : row["UnitPrice"],
                       "Quantity" : row["Quantity"],
                       "Discount" : row["Discount"]})

In [22]:
for index, row in mysql_query("""
    SELECT *
    FROM EmployeeTerritories
    """).iterrows():
    neodb.run("""
    MATCH (e:Employee), (t:Territory)
    WHERE e.EmployeeID = $EmployeeID AND t.TerritoryID = $TerritoryID
    CREATE (e)-[details:COVERS{
                        EmployeeID : $EmployeeID,
                        TerritoryID : $TerritoryID}]->(t)
    """, parameters = {"EmployeeID" : row["EmployeeID"],
                       "TerritoryID" : row["TerritoryID"]})

In [23]:
neodb.run("""
    MATCH (p:Product), (c:Category)
    WHERE p.CategoryID = c.CategoryID
    CREATE (p)-[:PART_OF]->(c)
    """)

neodb.run("""
    MATCH (p:Product), (s:Supplier)
    WHERE p.SupplierID = s.SupplierID
    CREATE (s)-[:SUPPLIES]->(p)
    """)

neodb.run("""
    MATCH (c:Customer), (o:Order)
    WHERE c.CustomerID = o.CustomerID
    CREATE (c)-[:PURCHASED]->(o)
    """)

neodb.run("""
    MATCH (e:Employee), (m:Employee)
    WHERE e.ReportsTo = m.EmployeeID
    CREATE (e)-[:REPORTS_TO]->(m)
    """)

neodb.run("""
    MATCH (r:Region), (t:Territory)
    WHERE r.RegionID = t.RegionID
    CREATE (r)-[:CONTAINS]->(t)
    """)

neodb.run("""
    MATCH (e:Employee), (o:Order)
    WHERE e.EmployeeID = o.EmployeeID
    CREATE (e)-[:SOLD]->(o)
    """)

neodb.run("""
    MATCH (s:Shipper), (o:Order)
    WHERE s.ShipperID = o.ShipVia
    CREATE (s)-[:SHIPPED]->(o)
    """)

In [24]:
pd.DataFrame(neodb.data("""
    MATCH (c:Category {CategoryName:"Produce"})<--(:Product)<--(s:Supplier)
    RETURN DISTINCT s.CompanyName as ProduceSuppliers
    """))

,ProduceSuppliers
0,Tokyo Traders
1,Grandma Kelly's Homestead
2,"G'day, Mate"
3,Mayumi's
4,Plutzer Lebensmittelgroßmärkte AG


In [39]:
pd.DataFrame(neodb.data("""
    MATCH (s:Supplier)-->(:Product)-->(c:Category)
    RETURN s.CompanyName as Company,
    collect(distinct c.CategoryName) as Categories
    """))

,Categories,Company
0,[Confections],Zaanse Snoepfabriek
1,"[Beverages, Condiments]",Exotic Liquids
2,"[Beverages, Confections]","Specialty Biscuits, Ltd."
3,[Dairy Products],Norske Meierier
4,[Beverages],Bigfoot Breweries
5,"[Produce, Condiments]",Grandma Kelly's Homestead
6,"[Condiments, Produce, Seafood]",Mayumi's
7,[Beverages],Refrescos Americanas LTDA
8,[Dairy Products],Formaggi Fortini s.r.l.
9,"[Confections, Condiments]",Forêts d'érables


In [26]:
pd.DataFrame(neodb.data("""
    MATCH (cust:Customer)-[:PURCHASED]->(:Order)-[o:INCLUDES]->(p:Product),
    (p)-[:PART_OF]->(c:Category {CategoryName:"Produce"})
    RETURN DISTINCT cust.ContactName as
    CustomerName, SUM(o.Quantity) AS TotalProductsPurchased
    """))

,CustomerName,TotalProductsPurchased
0,Martine Rancé,50.0
1,Miguel Angel Paolino,14.0
2,Art Braunschweiger,16.0
3,Maria Anders,17.0
4,Hari Kumar,72.0
5,Yoshi Latimer,15.0
6,Thomas Hardy,20.0
7,Frédérique Citeaux,30.0
8,Liu Wong,10.0
9,Carine Schmitt,3.0


In [6]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20,20)